# Derivation of Potential Field Source Surface Model with Connection to a Spacecraft

## Motivation
Due to the coronal magnetic field not being as strong as the photospheric field, in order to calculate likely field line configurations above the photosphere, we need to employ magnetic field models.

## Definition
By definition, a Potential Field Source Surface (PFSS) model provides an approximate description of the solar coronal magnetic field based on observed photospheric field (magnetograms). The main constraints of the model are stated in the name itself.

- Potential Field: By taking div B = 0, we consider a current-free magnetic field. This means there is no free energy, and the model is thus static.

- Source Surface: We assume that if a field line reaches a certain height above the photosphere (the source surface), the field line becomes open to the heliosphere, and the magnetic field is assumed to be purely radial. A typical value for the source surface is 2.5 solar radii.

For more information, see [here](https://ccmc.gsfc.nasa.gov/models/modelinfo.php?model=PFSS).

## Resources

- [JHelioviewer](http://swhv.oma.be/user_manual/#pfss), covered on the first exercise, allows for overplotting of a PFSS solution.
- This notebook allows for the creation of a PFSS solution for an given date, on Python

## Installation

1. Install Anaconda, a Python and Data Science package manager, for your OS
https://www.anaconda.com/products/individual


2. Use the environment.yml file to install relevant packages:
  - Open a terminal (Different for Mac (terminal), Linux (bash), Windows (anaconda terminal))

  - Generate a 'solarphysics' environment: 
        conda env create solarphysics

  - Install environment.yml file: 
        conda env update -n solarphysics --file environment.yml

  - Activate environment.yml file: 
        conda activate solarphysics

  - Open jupyter notebook: 
        jupyter notebook

  - Navigate to folder where you
  
3. Open the Notebook in Jupyter, either from the Anaconda GUI (Windows, MAC), or from the Anaconda Prompt:
jupyter notebook Notebookfile.ipynb  -->



## Begin by importing required packages

In [ ]:
!pip install pfsspy

In [ ]:
from datetime import datetime
import os

# Astropy for physical constants, unit and coordinate handling
import astropy.constants as const
import astropy.units as u
from astropy.coordinates import SkyCoord

import matplotlib.pyplot as plt
import numpy as np
import sunpy.map
import sunpy.io.fits

import pfsspy
import pfsspy.tracing as tracing
from pfsspy.sample_data import get_gong_map

### Download AIA map and save within imageResources folder

In [ ]:
if not os.path.exists('imageResources/Exercise_3/aia_map.fits'):
    import urllib.request
    urllib.request.urlretrieve(
        'http://jsoc2.stanford.edu/data/aia/synoptic/2020/09/01/H1300/AIA20200901_1300_0193.fits',
        'imageResources/Exercise_3/aia_map.fits')

aia = sunpy.map.Map('imageResources/Exercise_3/aia_map.fits')
dtime = aia.date

In [ ]:
ax = plt.subplot(1, 1, 1, projection=aia)
aia.plot(ax)

## Magnetic Field

### Initial magnetic field input

In [ ]:
gong_fname = get_gong_map()
gong_map = sunpy.map.Map(gong_fname)
# Remove the mean
gong_map = sunpy.map.Map(gong_map.data - np.mean(gong_map.data), gong_map.meta)

### PFSS Solution
# PFSS solution to solar magnetic field is calculated on a 3D grid (phi, s, rho).

#  rho = ln(r), and r is the standard spherical radial coordinate.
nrho = 25
rss = 2.5  # Source surface Radius
input = pfsspy.Input(gong_map, nrho, rss)

In [ ]:
# Plot the PFSS map
m = input.map
fig = plt.figure()
ax = plt.subplot(projection=m)
m.plot()
plt.colorbar()
ax.set_title('Input field')

### Create Magnetic field line footpoints (seeds)

We create a set of origin coordinates (field footpoints) that we can then derive the most likely magnetic field configuration from.

In [ ]:
# Create 5 points spaced between sin(lat)={0.35, 0.55}
s = np.linspace(0.35, 0.55, 5)
# Create 5 points spaced between long={60, 100} degrees
phi = np.linspace(60, 100, 5)
print(f's = {s}')
print(f'phi = {phi}')
# Make a 2D grid from these 1D points
s, phi = np.meshgrid(s, phi)

# Now convert the points to a coordinate object
lat = np.arcsin(s) * u.rad
lon = phi * u.deg
seeds = SkyCoord(lon.ravel(), lat.ravel(), 1.01 * const.R_sun,
                 frame=gong_map.coordinate_frame)

# Plot the field line footpoint map
m = input.map
fig = plt.figure()
ax = plt.subplot(projection=m)
m.plot()
plt.colorbar()

ax.plot_coord(seeds, color='black', marker='o', linewidth=0, markersize=2)

# Set the axes limits. These limits have to be in pixel values
# ax.set_xlim(0, 180)
# ax.set_ylim(45, 135)
ax.set_title('Field line footpoints')
ax.set_ylim(bottom=0)

## PFSS 
### Solution

In [ ]:
# Calculate PFSS Solution using input parameters
output = pfsspy.pfss(input)
tracer = tracing.PythonTracer()  # Trace field lines
flines = tracer.trace(seeds, output)  # Make use of footpoints (seeds)

### Field lines plotted on Magnetogram

In [ ]:
# Solution plotted on magnetogram
m = input.map
fig = plt.figure()
ax = plt.subplot(projection=m)
m.plot()
plt.colorbar()

for fline in flines:
    ax.plot_coord(fline.coords, color='black', linewidth=1)

# Set the axes limits. These limits have to be in pixel values
# ax.set_xlim(0, 180)
# ax.set_ylim(45, 135)
ax.set_title('Photospheric field and traced field lines')

### Overplotting on AIA observations

In [ ]:
fig = plt.figure()
ax = plt.subplot(1, 1, 1, projection=aia)
aia.plot(ax)
for fline in flines:
    ax.plot_coord(fline.coords, alpha=0.8, linewidth=1, color='white')

plt.show()